In [ ]:
import os
import json
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
# Загрузка prompts.json
with open("data/prompts.json", "r") as f:
    prompts = json.load(f)

# Преобразование в DataFrame для удобства анализа
df_prompts = pd.DataFrame(prompts)

# Предварительный просмотр данных
df_prompts.head()


In [ ]:
# Распределение стилей
style_counts = df_prompts["style"].value_counts()
plt.figure(figsize=(10, 5))
style_counts.plot(kind="bar")
plt.title("Распределение стилей")
plt.xlabel("Стиль")
plt.ylabel("Количество")
plt.show()


In [ ]:
# Объединяем все текстовые запросы для создания облака слов
all_prompts = " ".join(df_prompts["prompt"].tolist())

# Генерация облака слов
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(all_prompts)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Облако слов из текстовых запросов")
plt.show()


In [ ]:
IMAGE_DIR = "data/images"
PROCESSED_DIR = "data/processed_images"
os.makedirs(PROCESSED_DIR, exist_ok=True)

# Изменение размера изображений
for img_file in os.listdir(IMAGE_DIR):
    try:
        with Image.open(os.path.join(IMAGE_DIR, img_file)) as img:
            img_resized = img.resize((256, 256))  # Изменяем размер на 256x256
            img_resized.save(os.path.join(PROCESSED_DIR, img_file))
    except Exception as e:
        print(f"Ошибка при обработке {img_file}: {e}")


In [ ]:
# Привязка запросов к изображениям
df_prompts["image_path"] = df_prompts["id"].apply(lambda x: f"data/processed_images/{x}.png")

# Проверка наличия изображений
df_prompts["image_exists"] = df_prompts["image_path"].apply(lambda x: os.path.exists(x))

# Оставляем только те записи, где изображение существует
df_clean = df_prompts[df_prompts["image_exists"]]
df_clean.to_csv("data/processed_data.csv", index=False)

print(f"Сохранено {len(df_clean)} записей.")


In [ ]:
import torch

# Пример: Сохранение текстов и путей к изображениям для использования в обучении
processed_data = {
    "prompts": df_clean["prompt"].tolist(),
    "image_paths": df_clean["image_path"].tolist()
}

torch.save(processed_data, "data/processed_data.pt")
print("Данные сохранены в формате PyTorch.")


In [ ]:
import random

# Вывод нескольких изображений с запросами
sample_data = df_clean.sample(5)
for _, row in sample_data.iterrows():
    img = Image.open(row["image_path"])
    plt.figure()
    plt.imshow(img)
    plt.title(row["prompt"])
    plt.axis("off")
    plt.show()
